In [178]:
import pandas as pd

In [179]:
df_routes = pd.read_csv('routes.csv')
df_airports = pd.read_csv('airports.csv')



In [180]:
display(df_routes)

,index,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [181]:
df_routes = df_routes[['Source airport ID', 'Destination airport ID']]
df_airports = df_airports[['Airport ID','Latitude','Longitude']]

In [182]:
df_routes['Source airport ID'] = pd.to_numeric(df_routes['Source airport ID'], errors='coerce')
df_routes['Destination airport ID'] = pd.to_numeric(df_routes['Destination airport ID'], errors='coerce')
df_routes = df_routes.dropna()

df_airports['Airport ID'] = pd.to_numeric(df_airports['Airport ID'], errors='coerce')
df_airports['Latitude'] = pd.to_numeric(df_airports['Latitude'], errors='coerce')
df_airports['Longitude'] = pd.to_numeric(df_airports['Longitude'], errors='coerce')
df_airports = df_airports.dropna()

In [183]:
import pandas as pd
import numpy as np

# Função para calcular a distância
def calculate_distance(lat1, lon1, lat2, lon2):
    # Converter graus para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Aplicar a fórmula de Haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c
    return km

# Realizar o merge para obter as coordenadas do aeroporto de origem
df_routes = df_routes.merge(df_airports, how='left', left_on='Source airport ID', right_on='Airport ID')
df_routes.rename(columns={'Latitude': 'Source Latitude', 'Longitude': 'Source Longitude'}, inplace=True)
df_routes.drop(['Airport ID'], axis=1, inplace=True)

# Realizar o merge para obter as coordenadas do aeroporto de destino
df_routes = df_routes.merge(df_airports, how='left', left_on='Destination airport ID', right_on='Airport ID')
df_routes.rename(columns={'Latitude': 'Destination Latitude', 'Longitude': 'Destination Longitude'}, inplace=True)
df_routes.drop(['Airport ID'], axis=1, inplace=True)

# Calcular a distância e adicionar como nova coluna
df_routes['Distance'] = df_routes.apply(lambda row: calculate_distance(row['Source Latitude'], row['Source Longitude'], row['Destination Latitude'], row['Destination Longitude']), axis=1)

df_routes

,Source airport ID,Destination airport ID,Source Latitude,Source Longitude,Destination Latitude,Destination Longitude,Distance
0,2965.0,2990.0,43.449902,39.956600,55.606201,49.278702,1506.825604
1,2966.0,2990.0,46.283298,48.006302,55.606201,49.278702,1040.438320
2,2966.0,2962.0,46.283298,48.006302,44.225101,43.081902,448.164909
3,2968.0,2990.0,55.305801,61.503300,55.606201,49.278702,770.508500
4,2968.0,4078.0,55.305801,61.503300,55.012600,82.650703,1338.631467
...,...,...,...,...,...,...,...
67235,6334.0,3341.0,-33.058899,137.514008,-34.945000,138.531006,229.720619
67236,4029.0,2912.0,55.408798,37.906300,43.061298,74.477600,2942.819259
67237,2912.0,4029.0,43.061298,74.477600,55.408798,37.906300,2942.819259
67238,2912.0,2913.0,43.061298,74.477600,40.609001,72.793297,306.295375


In [184]:

df_airports = df_airports.dropna()

# Eliminar linhas duplicadas
df_routes['Route'] = df_routes.apply(lambda row: tuple(sorted([row['Source airport ID'], row['Destination airport ID']])), axis=1)
df_routes = df_routes[~df_routes.duplicated(subset='Route')]
df_routes.drop('Route', axis=1, inplace=True)

display(df_routes)


C:\Users\victor\AppData\Local\Temp\ipykernel_8556\3798130929.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_routes.drop('Route', axis=1, inplace=True)


,Source airport ID,Destination airport ID,Source Latitude,Source Longitude,Destination Latitude,Destination Longitude,Distance
0,2965.0,2990.0,43.449902,39.956600,55.606201,49.278702,1506.825604
1,2966.0,2990.0,46.283298,48.006302,55.606201,49.278702,1040.438320
2,2966.0,2962.0,46.283298,48.006302,44.225101,43.081902,448.164909
3,2968.0,2990.0,55.305801,61.503300,55.606201,49.278702,770.508500
4,2968.0,4078.0,55.305801,61.503300,55.012600,82.650703,1338.631467
...,...,...,...,...,...,...,...
67204,4320.0,3361.0,-32.794998,151.834000,-33.946098,151.177002,141.792318
67205,6793.0,3361.0,-33.381699,149.132996,-33.946098,151.177002,199.302583
67207,6317.0,3361.0,-33.131401,148.238998,-33.946098,151.177002,286.960962
67225,3361.0,6794.0,-33.946098,151.177002,-31.888599,152.514008,260.601236


In [185]:
# Eliminando todas as colunas desnecessárias
df_routes = df_routes.drop(['Destination Latitude', 'Destination Longitude'], axis=1)
df_routes = df_routes.drop(['Source Latitude', 'Source Longitude'], axis=1)
display(df_routes)

,Source airport ID,Destination airport ID,Distance
0,2965.0,2990.0,1506.825604
1,2966.0,2990.0,1040.438320
2,2966.0,2962.0,448.164909
3,2968.0,2990.0,770.508500
4,2968.0,4078.0,1338.631467
...,...,...,...
67204,4320.0,3361.0,141.792318
67205,6793.0,3361.0,199.302583
67207,6317.0,3361.0,286.960962
67225,3361.0,6794.0,260.601236


In [186]:
df_routes = df_routes.sort_values(by='Distance', ascending=False)
df_routes = df_routes.dropna()
df_routes = df_routes.drop(df_routes.index[-1])
df_routes['Distance'] = df_routes['Distance'].round(2)
display(df_routes)


,Source airport ID,Destination airport ID,Distance
44962,907.0,5613.0,16082.26
44966,910.0,5613.0,15937.11
6807,3361.0,3670.0,13808.18
20037,3682.0,813.0,13582.59
13877,2188.0,3484.0,13400.08
...,...,...,...
1746,9739.0,9744.0,11.03
3006,7194.0,7187.0,10.34
1741,9744.0,3599.0,9.15
65783,5490.0,5543.0,9.11


In [187]:
class Grafo:
    def __init__(self):
        self.vertices = set()
        self.arestas = {}
        self.vizinhos = {}

    def adiciona_aresta(self, origem, destino, peso):
        if origem not in self.vertices:
            self.vertices.add(origem)
            self.vizinhos[origem] = []
        if destino not in self.vertices:
            self.vertices.add(destino)
            self.vizinhos[destino] = []
        
        self.arestas[(origem, destino)] = peso
        self.arestas[(destino, origem)] = peso  # Garante o grafo não direcional
        self.vizinhos[origem].append((destino, peso))
        self.vizinhos[destino].append((origem, peso))

In [188]:
grafo = Grafo()
for i, row in df_routes.iterrows():
    grafo.adiciona_aresta(row['Source airport ID'], row['Destination airport ID'], row['Distance'])

In [189]:
def bellman_ford(grafo, origem, destino):
    distancias = {vertice: float('inf') for vertice in grafo.vertices}
    antecessor = {vertice: None for vertice in grafo.vertices}

    distancias[origem] = 0

    for _ in range(len(grafo.vertices) - 1):
        for (u, v), peso in grafo.arestas.items():
            if distancias[u] + peso < distancias[v]:
                distancias[v] = distancias[u] + peso
                antecessor[v] = u

    # Reconstrói o caminho mais curto
    caminho = []
    atual = destino
    while atual is not None:
        caminho.append(atual)
        atual = antecessor[atual]
    caminho = caminho[::-1]  # Inverte o caminho

    if caminho[0] == origem:
        return caminho, [distancias[v] for v in caminho]
    else:
        return "Não existe caminho", []



In [190]:
# escolha qualquer Source airport ID, como origem.  
# escolha qualquer Destination airport ID, como destino.

display(df_routes)

,Source airport ID,Destination airport ID,Distance
44962,907.0,5613.0,16082.26
44966,910.0,5613.0,15937.11
6807,3361.0,3670.0,13808.18
20037,3682.0,813.0,13582.59
13877,2188.0,3484.0,13400.08
...,...,...,...
1746,9739.0,9744.0,11.03
3006,7194.0,7187.0,10.34
1741,9744.0,3599.0,9.15
65783,5490.0,5543.0,9.11


In [194]:
# Exemplo de uso
while True:
    origem = int(input('ID do aeroporto de origem: '))
    if (df_routes['Source airport ID'] == origem).any():
        print('Esse ID do aeroporto não existe. Digite outro')
        break

while True:
    destino = int(input('ID do aeroporto de destino: '))
    if (df_routes['Destination airport ID'] == destino).any():
        print('Esse ID do aeroporto não existe. Digite outro')
        break

caminho, distancias = bellman_ford(grafo, origem, destino)
print("Caminho:", caminho)
print("Distâncias:", distancias)


# o algoritmo Bellman-Ford encontrará o menor caminho entre esses dois aeroportos em KM!

Esse ID do aeroporto não existe. Digite outro
Esse ID do aeroporto não existe. Digite outro
Caminho: [50.0, 187.0, 160.0, 178.0, 2279.0, 5.0, 3]
Distâncias: [0, 261.86, 1264.69, 2456.12, 10401.849999999999, 15479.609999999999, 15994.909999999998]
